## Generating instruction-answer pairs with Gemini

References:

* https://ai.google.dev/tutorials/python_quickstart
* https://github.com/deep-diver/llmops-pipeline/issues/8#issue-2212315206

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        if "gemini-1.5" in m.name:
            print(m.name)

In [ ]:
model_name = "gemini-1.5-pro-latest"

model = genai.GenerativeModel(model_name)

In [ ]:
prompt = """
Generate a series of (instruction, response) pairs that are similar in context and structure to the example provided below. Each pair should consist of a concise instruction followed by an appropriate, detailed response. The instruction should pose a clear task or question, while the response should provide a comprehensive answer or solution that could be understood by someone with a basic understanding of the subject.

Example pair:

Instruction: This Java function below is not working as I expected, it's supposed to go through an array of numbers and if it finds a number in the array that is divisible by 2 with a remainder of 0, then it adds it to a new array and returns the new array at the end. Can you help debug the function and explain how why it's not working as expected?\n\npublic static List checkForEvens(int [] nums) {\n ArrayList anwer = new ArrayList<Integer>();\n for(int i = 0; i < nums.length; i++) {\n if(nums[i] / 2 == 0) {\n answer.add(nums[i]\n } \n }\n}
Response: Certainly! The function 'checkForEvens' iterates through an array and if a number in the array is divisible by 2, then it gets added to 'answer' which is later returned.\n\nThe reason the function may not be returning the expected result is because of the if statement, 'if (nums[i] /2 == 0)'. The statement is looking for numbers that get divided by 2 in the array and equal 0, which doesn't align with what you expect the function to do. \n\nInstead, you want to use the modulo operator to check for the remainder of a number that is being divided by 2. If a number is divided by 2 and returns a remainder of 0, then we can assume that it's an even number. So, the small change of 'if (nums[i] % 2 == 0)' should solve your issue. \n\nI've rewritten the function below with the correction.\n\npublic static List checkForEvens(int [] nums) {\n ArrayList anwer = new ArrayList<Integer>();\n for(int i = 0; i < nums.length; i++) {\n if(nums[i] % 2 == 0) {\n answer.add(nums[i]\n } \n }\n}

Your task is to generate more pairs that maintain this level of clarity and detail. The topic is Coding. Ensure that the responses are informative and accurate, suitable for an educational context.

Store the generated pairs in JSON format, with each pair as an object within an array. Each object should have two key-value pairs: "instruction" and "response". For instance:

[{"instruction": text, "response": text}, {"instruction": text, "response": text}, ...]

Remember to maintain consistency in the format and ensure the generated pairs are diverse and cover a broad range of subjects. You must return the response
in the asked format and you must to add any additional text in your response.
"""

In [ ]:
response = model.generate_content(prompt)

In [ ]:
from IPython.display import Markdown
import textwrap

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

to_markdown(response.text)

In [ ]:
eval(response.text)

## Generating similar responses with periodic hints from the original dataset

In [ ]:
!pip install datasets -q -U

In [ ]:
from datasets import load_dataset

existing_dataset = load_dataset("sayakpaul/no_robots_only_coding", split="train_sft")
existing_dataset

In [ ]:
existing_dataset[1]["messages"]

In [ ]:
import numpy as np

num_periods = 5
total_original_samples = len(existing_dataset)
random_indieces = np.random.randint(0, total_original_samples, size=(num_periods))
random_indieces

In [ ]:
from typing import List, Dict

def generate_with_gemini():
    response = model.generate_content(prompt)
    return eval(response.text)

def format_response(responses: List[Dict[str, str]]):
    final_instruction_answer_pair = []

    for response in responses:
        user_response_dict = assistant_response_dict = {}
        user_response_dict["content"] = response["instruction"]
        user_response_dict["role"] = "user"
        assistant_response_dict["content"] = response["response"]
        assistant_response_dict["role"] = "assistant"

        final_instruction_answer_pair.append([user_response_dict, assistant_response_dict])

    return final_instruction_answer_pair

In [ ]:
# demo
demo_response_from_gemini = generate_with_gemini()
demo_final_responses = format_response(demo_response_from_gemini)
demo_final_responses[0]